In [1]:
!pip install cython
!pip install gensim==3.4.0

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB 435.7 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.8 MB 751.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.8 MB 1.3 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.8 MB 2.2 MB/s eta 0:00:02
   -------------- ------------------------- 1.0/2.8 MB 3.9 MB/s eta 0:00:01
   ---------------------------- ----------- 2.0/2.8 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/22.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/22.2 MB ? eta -:--:--
     --------------------------------------- 0.0/22.2 MB 393.8 kB/s eta 0:00:57
     ---------------------------------------- 0.3/22.2 MB 1.8 MB/s eta 0:00:13
     - ----------------

In [2]:
##year_1 and year_2 comprise the time period of ngrams to read##
year_1=2000
year_2=2012

import numpy
import scipy
import cython
import gensim
from gensim import models, similarities
import logging
import os
import re
import sys  
import itertools
import math


c:\Users\mateo\anaconda3\envs\Ngram\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:

#reload(sys)  
#sys.setdefaultencoding('utf8')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname, start_year, end_year, limit=None):
        self.dirname = dirname
        self.start_year = start_year
        self.end_year = end_year
        self.limit = limit
 
    def __iter__(self):
         # iterate through each the compressed file directory
        for fname in os.listdir(self.dirname):
            # for each compressed file open it
            with gensim.utils.smart_open(os.path.join(self.dirname, fname)) as fin:
                for line in itertools.islice(fin, self.limit):
                    line = gensim.utils.to_unicode(line).split("\t")
                    if len(line)<3:
                        continue
                    ngram = line[0]
                    try:
                        year = int(line[1])
                    except ValueError:
                        continue
                    match_count = int(line[2])
                    if year < self.start_year or year > self.end_year:
                        continue
                    # lower case the ngram, remove pos
                    processed_ngram = [word.split("_")[0] for word in ngram.lower().split()]
                    for x in range(match_count):
                        yield processed_ngram

assert gensim.models.word2vec.FAST_VERSION > -1
##Reads ngrams one-by-one into word2vec##
sentences = MySentences("ngrams/googlebooks-eng-all-5gram-20120701-0/", year_1, year_2) # a memory-friendly iterator

###Set parameters. Details here: https://radimrehurek.com/gensim/models/word2vec.html###
model = gensim.models.word2vec.Word2Vec(sentences=sentences ,sg=1, size=300, window=5, min_count=10, workers=10, hs=0, negative=8)
model.save('w2vmodel_ng5_'+str(year_1)+'_'+str(year_2)+'_full')
syn0_object=model.wv.syn0

##output vector space##
numpy.savetxt('syn0_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt',syn0_object,delimiter=" ")

#output vocab list#
vocab_list = model.wv.index2word
for i in range(0,len(vocab_list)):
	if vocab_list[i] == '':
		vocab_list[i] = "thisisanemptytoken"+str(i)

with open('vocab_list_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt','wb') as outfile:
    for i in range(0,len(vocab_list)):
        outfile.write(vocab_list[i].encode('utf8')+"\n".encode('ascii'))


2024-04-05 21:18:37,516 : INFO : collecting all words and their counts
2024-04-05 21:18:37,517 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-05 21:18:37,537 : INFO : PROGRESS: at sentence #10000, processed 50000 words, keeping 114 word types
2024-04-05 21:18:37,553 : INFO : PROGRESS: at sentence #20000, processed 100000 words, keeping 233 word types
2024-04-05 21:18:37,574 : INFO : PROGRESS: at sentence #30000, processed 150000 words, keeping 410 word types
2024-04-05 21:18:37,600 : INFO : PROGRESS: at sentence #40000, processed 200000 words, keeping 550 word types
2024-04-05 21:18:37,630 : INFO : PROGRESS: at sentence #50000, processed 250000 words, keeping 785 word types
2024-04-05 21:18:37,905 : INFO : PROGRESS: at sentence #60000, processed 300000 words, keeping 975 word types
2024-04-05 21:18:37,932 : INFO : PROGRESS: at sentence #70000, processed 350000 words, keeping 1083 word types
2024-04-05 21:18:37,957 : INFO : PROGRESS: at sentence #8000

In [ ]:
import os
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from collections import Counter
import itertools

# Placeholder for the data directory and years
dirname = "/ngrams/googlebooks-eng-all-5gram-20120701-0/googlebooks-eng-all-5gram-20120701-0/"
start_year = 2000
end_year = 2012

# Function to read and preprocess n-grams
def read_ngrams(dirname, start_year, end_year, limit=None):
    vocab = Counter()
    data = []
    for fname in os.listdir(dirname):
        with open(os.path.join(dirname, fname), 'r') as fin:
            for line in itertools.islice(fin, limit):
                parts = line.strip().split("\t")
                if len(parts) < 3:
                    continue
                ngram, year, match_count = parts[0], int(parts[1]), int(parts[2])
                if start_year <= year <= end_year:
                    words = [word.split("_")[0].lower() for word in ngram.split()]
                    vocab.update(words)
                    data.append(words)
    return data, vocab

data, vocab = read_ngrams(dirname, start_year, end_year)

# Function to build a vocabulary index
def build_vocab_index(vocab):
    vocab_index = {word: i for i, (word, _) in enumerate(vocab.most_common(), 1)}
    return vocab_index

vocab_index = build_vocab_index(vocab)

# Function to convert words to indices
def words_to_indices(data, vocab_index):
    indexed_data = []
    for sentence in data:
        indexed_sentence = [vocab_index[word] for word in sentence if word in vocab_index]
        indexed_data.append(indexed_sentence)
    return indexed_data

indexed_data = words_to_indices(data, vocab_index)
